# Urban Pulse - Data Preprocessing

## Data Cleaning and Feature Engineering

This notebook handles:
- Missing value imputation
- Outlier detection and handling
- Datetime parsing and temporal feature creation
- Derived feature engineering (rush hour, traffic stress levels)
- Data quality documentation


In [10]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Setup project path to import from src
from setup_path import setup_project_path
setup_project_path()

from src.data_processing import (
    load_data,
    inspect_data,
    handle_missing_values,
    handle_outliers,
    parse_datetime,
    create_rush_hour_feature,
    create_traffic_stress_level,
    preprocess_pipeline,
    load_and_clean_data
)

print("✓ Libraries imported successfully")


✓ Libraries imported successfully


## 1. Load Raw Data

Load the dataset from the exploration notebook or directly.


In [11]:
# Load raw data
data_path = '../data/raw/Metro_Interstate_Traffic_Volume.csv'

try:
    df_raw = load_data(data_path)
    print(f"✓ Raw data loaded successfully: {df_raw.shape}")
except FileNotFoundError:
    print(f"⚠️  Data file not found!")
    print(f"Expected file location: {data_path}")
    print("Please ensure the CSV file is in the data/raw/ directory.")


✓ Data loaded successfully: 48204 rows, 9 columns
✓ Raw data loaded successfully: (48204, 9)


## 2. Run Complete Preprocessing Pipeline

The preprocessing pipeline handles all cleaning and feature engineering steps automatically.


### 2.1 Missing Value Handling Strategy - Justification

**Forward Fill for numeric/time series columns:**
- Time series data: Previous hour's value is realistic for missing weather/traffic data
- Preserves all 48,204 observations
- Maintains time continuity

**Special handling for holiday column:**
- 99.87% missing values (48,143/48,204)
- Use **mode** ("None") instead of forward fill
- Reason: Holidays are single-day events; forward fill would incorrectly mark subsequent days as holidays


### 2.2 Outlier Handling Strategy - Justification

**Why Cap outliers (IQR method):**
- Preserves all 48,204 observations
- Maintains time series continuity (no gaps)
- Caps extreme values to reasonable limits while keeping data points

**Why not remove:**
- Would reduce dataset size and break hourly time series structure


In [12]:
# Run complete preprocessing pipeline
df_processed, preprocessing_report = preprocess_pipeline(
    df_raw,
    target_column='traffic_volume',
    date_column='date_time',
    missing_strategy='forward_fill',  # Good for time series data
    outlier_method='cap'  # Cap outliers rather than remove
)


STARTING DATA PREPROCESSING PIPELINE
DATA QUALITY REPORT
Shape: 48204 rows × 9 columns

Missing Values:
  holiday: 48143 (99.87%)

Duplicate Rows: 17
Memory Usage: 11.71 MB
✓ Parsed datetime column 'date_time' and extracted temporal features

Handling missing values using 'forward_fill' strategy...
  holiday: 48143 -> 0 missing values
✓ Capped outliers in 'traffic_volume' at [-4417.00, 10543.00]
✓ Created rush hour features
✓ Created traffic stress levels:
  Low: < 2158
  Medium: 2158 - 4586
  High: >= 4586
DATA QUALITY REPORT
Shape: 48204 rows × 19 columns

Missing Values:

Duplicate Rows: 17
Memory Usage: 16.99 MB

PREPROCESSING PIPELINE COMPLETE
Initial rows: 48204
Final rows: 48204
Features created: 10


## 3. Verify Preprocessing Results

Check that all features were created correctly.


In [19]:
# Display new features created
print("New Features Created:")
print("="*60)
new_features = ['year', 'month', 'day', 'hour', 'day_of_week', 'is_weekend',
                'is_rush_hour', 'rush_hour_type', 'traffic_stress_level', 'is_congested']

for feature in new_features:
    if feature in df_processed.columns:
        print(f"✓ {feature}")
        if df_processed[feature].dtype == 'object':
            print(f"    Values: {df_processed[feature].value_counts().to_dict()}")
        else:
            print(f"    Range: {df_processed[feature].min()} - {df_processed[feature].max()}")


New Features Created:
✓ year
    Range: 2012 - 2018
✓ month
    Range: 1 - 12
✓ day
    Range: 1 - 31
✓ hour
    Range: 0 - 23
✓ day_of_week
    Range: 0 - 6
✓ is_weekend
    Range: 0 - 1
✓ is_rush_hour
    Range: 0 - 1
✓ rush_hour_type
    Values: {'normal': 36147, 'morning_rush': 6177, 'evening_rush': 5880}
✓ traffic_stress_level
    Values: {'Medium': 16387, 'High': 15910, 'Low': 15907}
✓ is_congested
    Range: 0 - 1


## 4. Save Processed Data

Save the cleaned and processed dataset for use in EDA and ML notebooks.


In [20]:
# Save processed data
output_path = '../data/processed/traffic_cleaned.csv'
df_processed.to_csv(output_path, index=False)
print(f"✓ Processed data saved to: {output_path}")
print(f"  Shape: {df_processed.shape}")


✓ Processed data saved to: ../data/processed/traffic_cleaned.csv
  Shape: (48204, 19)
